# Association analysis brief overview

Association analysis is relatively straightforward concept. In addition, it is an unsupervised learning tool that looks for  patterns in data so there is relatively low requirement for initial data analysis and domain expertise. 

Definitions (ref Stanford CS246:Mining Massive Data Sets):

**Support** is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

**Confidence** is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

**Lift** is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

In [2]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from IPython.display import display, HTML


# in this case I am not using hand-coded a prirori, but we will code it for 
# study purposes later in the notebook (stay tuned :) )
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
# load the data
raw_df = pd.read_csv('export_2017.csv', encoding = "utf-8")#, nrows = 100000)
raw_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID,...,ART_C_PROD_NUMBER,ART_V_ART_DESCRIPTION,ART_V_ART_DESCRIPTION_1,ART_V_PG_LEVEL3_DESC,ART_V_PG_LEVEL4_DESC,ART_V_PG_LEVEL5_DESC,ART_V_PG_LEVEL6_DESC,ART_V_MANUFACTURER_DESC,PPR_ART_CLASS_DESC,PPR_N_LEVEL2_ID
0,341999621,80391417,0,06-JAN-17,1311010,62776372,12378259,1.00,61312,19997,...,139428,139428.16 Royal Canin British Shorthair Adult ...,139428.16 Royal Canin British Shorthair Adult ...,Katzenfutter trocken,Royal Canin Rasse (Breed),British Shorthair,British Shorthair,Royal Canin,Royal Canin,3
1,348334117,81849313,0,02-FEB-17,2031175,111951408,2913095,1.00,70828,48693,...,523432,"523432.4 Bozita Original - 12 kg + 1,8 kg gratis","523432.4 Bozita Original - 12 kg + 1,8 kg gratis",Hundefutter trocken,Bozita,Bozita,Bozita,Doggy-Bozita,Dry Food Cat / Dog,2
2,348334114,81849313,0,02-FEB-17,1568093,53626549,2913095,1.00,56851,43068,...,468126,468126.4 Sparpaket Lukullus feine Kauknochen 3...,468126.4 Sparpaket Lukullus feine Kauknochen 3...,Hundesnacks,★ Lukullus,Lukullus feine Kauknochen,Lukullus feine Kauknochen,MATINA,snacks,2
3,354903151,83341895,0,02-MAR-17,1910589,38614848,7097883,0.25,51010,39043,...,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3
4,354903151,83341895,0,02-MAR-17,1910589,38614828,7097883,0.25,51174,39043,...,445245,445245.29 Sparpaket Animonda Carny Adult 24 x ...,445245.29 Sparpaket Animonda Carny Adult 24 x ...,Katzenfutter nass,Animonda Carny,Carny Fleisch,Carny Fleisch,Animonda,Canned Food Cat / Dog,3


In [15]:
# let us remove unsued columns:
col_list = ['CUS_N_KEY_CUSTOMER', 'ART_C_PROD_NUMBER' , 'SCM_D_QUANTITY']
df = raw_df[col_list]
df['CUS_N_KEY_CUSTOMER'] = df['CUS_N_KEY_CUSTOMER'].astype('str')

C:\Users\vladyslavp\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [24]:
df.groupby(['CUS_N_KEY_CUSTOMER', 'ART_C_PROD_NUMBER']).sum().reset_index()

,CUS_N_KEY_CUSTOMER,ART_C_PROD_NUMBER,SCM_D_QUANTITY
0,10000002,9650,1.0000
1,10000002,48072,1.0000
2,10000002,136559,1.0000
3,10000002,199688,2.0000
4,10000002,199692,1.0000
5,10000002,199694,1.0000
6,10000002,218239,1.0000
7,10000002,461481,2.0000
8,10000002,462624,2.0000
9,10000002,476490,1.0000


In [25]:
df_pivot = df.pivot(index='CUS_N_KEY_CUSTOMER', columns='ART_C_PROD_NUMBER', values='SCM_D_QUANTITY')

IndexError: index 216624907 is out of bounds for axis 0 with size 216623688